In [ ]:
%pip install semantic-link
import sempy.fabric as fabric
import pyspark.sql.functions as f
import time

In [ ]:
workspace = 'Sneaker Workspace'
semantic_model = 'Nike Sneaker Calendar'
table = 'Launches'

delta_table_name = 'launches_export'

In [ ]:
refresh_history = fabric.list_refresh_requests(workspace=workspace, dataset=semantic_model)
refresh_history_spark = spark.createDataFrame(refresh_history)
refresh_history_spark = refresh_history_spark.withColumn( \
        'Refresh Duration', \
        (f.col('End Time').cast('long') - f.col('Start Time').cast('long'))
    )
refresh_average_duration = refresh_history_spark.agg({'Refresh Duration': 'avg'}).first()[0]
time_to_sleep = refresh_average_duration / 5

In [ ]:
table_dictionary = [{'table': table}]
refresh_request_id = fabric.refresh_dataset(workspace=workspace, \
                                            dataset=semantic_model, \
                                            objects=table_dictionary)

In [ ]:
loop_count = 0
refresh_status = ''
exit_statuses = ['Completed','Failed']
while loop_count < 6:
    refresh_status = fabric.get_refresh_execution_details(workspace=workspace,dataset=semantic_model,refresh_request_id=refresh_request_id).status
    if refresh_status in exit_statuses:
        break
    time.sleep(time_to_sleep)
    ++loop_count
print(refresh_status)

In [ ]:
table_fabric_df = fabric.read_table(workspace=workspace, dataset=semantic_model, table=table)
table_spark_df = spark.createDataFrame(table_fabric_df).select( \
                    f.col('Release Month').alias('release_month') \
                    , f.col('Release Day').alias('release_day') \
                    , f.col('Shoe Model').alias('shoe_model') \
                    , f.col('Shoe Name').alias('shoe_name'))
table_spark_df.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)